In [44]:
import os
from dotenv import find_dotenv, load_dotenv
_=load_dotenv(find_dotenv())
GROQ_API_KEY=os.environ['GROQ_API_KEY']

### Completion model


In [45]:
from langchain_groq import ChatGroq
llmModel = ChatGroq(model="llama-3.3-70b-versatile")

# This is for completions model
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} story about {topic}"
)

llmModelPrompt = prompt_template.format(
    adjective="curious",
    topic="Tesla"
)

res = llmModel.invoke(llmModelPrompt)
print(res)

content='It was a chilly winter evening in 1895 when Nikola Tesla, the renowned inventor and engineer, arrived at his laboratory in New York City. As he began to work on his latest experiment, he noticed something peculiar. His equipment, which included a variety of electrical devices and machinery, seemed to be behaving erratically.\n\nTesla, being a man of immense curiosity and dedication to his work, decided to investigate the cause of the disturbance. He spent hours checking and rechecking his equipment, but to no avail. The machines continued to malfunction, and Tesla was at a loss for an explanation.\n\nJust as he was about to give up for the night, Tesla received an unexpected visit from his friend and fellow inventor, George Westinghouse. Westinghouse had stopped by to discuss some business matters, but as he entered the laboratory, he noticed the strange occurrences.\n\n"Tesla, what\'s going on here?" Westinghouse asked, eyeing the malfunctioning equipment. "It looks like your

## chat completion model


In [46]:
from langchain_groq import ChatGroq
chatModel=ChatGroq(model='llama-3.1-8b-instant')
from langchain_core.prompts import ChatPromptTemplate
chat_template=ChatPromptTemplate.from_messages(
  [  
      ('system', 'You are an {profession} expert on {topic}'),
    ('human', 'Hello, Mr. {profession}, can you please answer a question?'),
    ('ai', 'Sure!'),
    ('human', '{user_input}')
    ]
)
messages=chat_template.format_messages(
    profession='Historian',
    topic='The Kennedy Family',
    user_input='How many grandchildren had Joseph P. Kennedy?'
)
response=chatModel.invoke(messages) 

print(response.content)

Joseph P. Kennedy, the patriarch of the Kennedy family, had a total of 9 grandchildren.


## Few Shot Prompting & Chains


In [47]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

examples = [
    {"input": "hi!", "output": "¡hola!"},
    {"input": "bye!", "output": "¡adiós!"},
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an English-Spanish translator."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

chain=final_prompt | chatModel
res=chain.invoke({"input": "How are you?"})
print(res.content)

Estoy bien, gracias. ¿Y tú? (I'm fine, thanks. And you?)


In [48]:
res

AIMessage(content="Estoy bien, gracias. ¿Y tú? (I'm fine, thanks. And you?)", response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 80, 'total_tokens': 101, 'completion_time': 0.020541844, 'prompt_time': 0.004267606, 'queue_time': 0.052553834, 'total_time': 0.02480945}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_7b3cfae3af', 'finish_reason': 'stop', 'logprobs': None}, id='run-e99872ed-b0d4-4352-8c7f-fafc207244ac-0', usage_metadata={'input_tokens': 80, 'output_tokens': 21, 'total_tokens': 101})

## Output Parsers

In [49]:
from langchain_groq import ChatGroq
llmModel = ChatGroq(model="llama-3.3-70b-versatile")

# This is for completions model
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers.json import SimpleJsonOutputParser

json_prompt=PromptTemplate.from_template(
        "Return a JSON object with an `answer` key that answers the following question: {question}"
)
json_parser=SimpleJsonOutputParser()
json_chain=json_prompt | llmModel | json_parser


In [50]:
json_parser.get_format_instructions()

'Return a JSON object.'

In [51]:
res=json_chain.invoke({"question": "What is the biggest country?"})
res

{'answer': 'Russia'}

In [52]:
type(res)

dict

Optionally, you can use Pydantic to define a custom output format


In [53]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

In [54]:
# Define a Pydantic Object with the desired output format.
class Joke(BaseModel):
    setup:str = Field(description="Question to set up a joke.")
    punchline:str=Field(description="answer to resolve the joke")

In [55]:
parser = JsonOutputParser(pydantic_object=Joke)

# Add the parser format instructions in the prompt definition.
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# Create a chain with the prompt and the parser
chain = prompt | chatModel | parser

chain.invoke({"query": "Tell me a joke."})

{'setup': "Why couldn't the bicycle stand up by itself?",
 'punchline': 'Because it was two-tired.'}